Creating input-target pairs.

Sliding window approach is used here.

Libraries_used

In [5]:
import importlib
import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader

Instantiate BPE tokenizer from tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
with open("../Required_files/the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Removing first 50 Tokens

In [ ]:
enc_sample = enc_text[50:]

Context size

It determines how many tokens are to be included in thw input

In [ ]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print("x is {}".format(x))
print("y is      {}".format(y))


x is [290, 4920, 2241, 287]
y is      [4920, 2241, 287, 257]


Sequence in which LLM gets trained for each input

In [ ]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
        
    print(context,"---->",desired)
    print(tokenizer.decode(context),"---->",tokenizer.decode([desired]))

[290] ----> 4920
 and ---->  established
[290, 4920] ----> 2241
 and established ---->  himself
[290, 4920, 2241] ----> 287
 and established himself ---->  in
[290, 4920, 2241, 287] ----> 257
 and established himself in ---->  a




Pytorch dataset and dataloader are used to create provision for training dataset in batches

Create a dataset from text file

In [6]:
class GPTDataserV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids = []
        self.taget_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

        # Use sliding window to chink the book inot overlapping seqeunces

        for i in range(0,len(token_ids) - max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.ouput_ids.append(torch.tensor(target_chunk))
                                  
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
       return self.input_ids[idx],self.target_ids[idx]
            

Function for dataloader

In [8]:
def create_dataloader_v1(txt,batch_size = 4, max_length =256,
                          stride = 128, shuffle =True, drop_last =True,num_workers = 0):
    
    # Intialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create Dataset
    dataset = GPTDataserV1(txt,tokenizer,max_length,stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle= shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader



                         